# Sign Language Detection Using Action Recognition

Using Nicholas Renotte's [Sign Language Detection using ACTION RECOGNITION with Python | LSTM Deep Learning Model](https://youtu.be/doDUihpj6ro) tutorial

[NR's code on GitHub](https://github.com/nicknochnack/ActionDetectionforSignLanguage)

## Goal: Real-time sign language detection using sequences

1. Estract holistic keypoints
2. Train an LSTM DL model
3. Make real-time predictions using sesquences

We will use:
* __media-pipe holistic__ to extract __keypoints__ from hand, body, face
* __TensorFlow__ and __Keras__ to build up a __long short-term memory deep learning model__ (__LSTM DL__) for predictions

## How it works

1. collect keypoints from mediapipe holistic
2. train a deep neural network with LSTM layers for sequences
3. perform real-time sign language detection using OpenCV

## 1. Install and import dependencies

In [ ]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib
# not installing tensorflow-gpu since I'm not using GPU

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## 2. Find and view keypoints using MediaPipe (MP) holistic

We will be using the mediapipe package to create the following variables:

* __mp_holistic__ will be used to make our detections
* __mp_drawing__ will be used to draw our detections

In [2]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [3]:
# mediapipe_detection(image, model) will find the mediapipe landmarks for an image
# image: the image from the feed we will be scrutinizing
# model: the MP detection model
def mediapipe_detection(image, model):
    # convert color from BGR (cv) to RGB (for mp detection)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # make image unwriteable to save memory
    image.flags.writeable = False
    # make prediction using the MP detection model
    results = model.process(image)
    # make image writeable again
    image.flags.writeable = True
    # convert image back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [ ]:
# FYI: for information on the cv2.cvtColor() function
cv2.cvtColor??

In [4]:
# draw_landmarks(image, results) will draw the landmark points over the live image
# image: the image from the feed we will be drawing on top of
# results: the landmark list we found with mediapipe_detection()
def draw_landmarks(image, results):
    # draw face landmarks & connections
    # no longer use FACE_CONNECTIONS.  Rather, use FACEMESH_TESSELATION
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    # draw pose landmarks & connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    # draw left hand landmarks & connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    # draw right hand landmarks & connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    

In [ ]:
# FYI: media pipe's documentation on mp_drawing.draw_landmarks
# (mp.solutions.drawing_utils.draw_landmarks)
mp_drawing.draw_landmarks??

In [163]:
# shows what landmark is connected to what landmark
# nose = 0, 1 = inner left eye, 4 = inner right eye, 2 = left eye, 3 = outer left eye
mp_holistic.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [5]:
cap = cv2.VideoCapture(2) # 2 is for logitech (near right USB-C port)
# set the mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        #read feed
        ret, frame = cap.read()
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        #draw landmarks and connections
        draw_landmarks(image, results)
        # show to screen
        cv2.imshow('OpenCV Feed', image)
        #break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    # end the video capture setting
    cap.release()
    cv2.waitKey(1)
    # close the video capture window(s)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
#this will show the number of landmark points of the left hand in the ending frame.
len(results.face_landmarks.landmark)

In [ ]:
draw_landmarks(frame, results)

In [ ]:
# plot the last frame from the frame array
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
results.face_landmarks

In [ ]:
frame[:2]

In [164]:
# data points for image detection vs landmark detection
print('one frame... ', 'image:', len(frame) * len(frame[0]) * 16 * 2, 'landmark:', 468 * 6 * 16 * 2)
print('two seconds @ 15fps... ', 'image:', len(frame) * len(frame[0]) * 16 * 2, 'landmark:', 468 * 6 * 16 * 2)

one frame...  image: 66355200 landmark: 89856
two seconds @ 15fps...  image: 66355200 landmark: 89856


## 3. Extract keypoint values

### input data

The input data used for this action detection model is a series of __30 arrays__.  __Each__ array contains __1662 values__ (30, 1662).

__Each__ array represents a __single frame__'s the landmark values (1662 values).

We will be using __30 frames of action__ to train and test on.

Typical landmarks will look like this:
<div>
    </div>
* We will concatenate the landmarks into a numpy array.
* If there are no landmarks then we will return a numpy 0 array.

In [165]:
pose = []
for res in results.pose_landmarks.landmark:
    landmark_data = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(landmark_data)

In [166]:
# rewrite above code onto one line:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark])


In [ ]:
len(pose) # should be 33 since there are 33 mp pose landmarks

In [ ]:
pose[:5]

In [ ]:
pose.shape

In [ ]:
pose.flatten()

In [ ]:
pose.flatten().shape

In [167]:
# save the number of mp landmarks for:
# facemesh, pose, left hand, right hand
# numbers found in MP documentation
num_mp_lmks = {
    'face' : {'num' : 468, 'dim' : 3}, # dim: x,y,z
    'pose' : {'num' : 33, 'dim' : 4}, # dim: x,y,z,visibility
    'hand' : {'num' : 21, 'dim' : 3} # dim: x,y,z
}
# add total by multiplying number of landmarks w/ their dimensions
for v in num_mp_lmks.values():
    v['total'] = (v['num'] * v['dim'])

In [168]:
num_mp_lmks

{'face': {'num': 468, 'dim': 3, 'total': 1404},
 'pose': {'num': 33, 'dim': 4, 'total': 132},
 'hand': {'num': 21, 'dim': 3, 'total': 63}}

In [169]:
len(np.zeros(num_mp_lmks['pose']['total']))

132

In [170]:
pose = np.array([[res.x, res.y, res.z, res.visibility]\
        for res in results.pose_landmarks.landmark]).flatten()\
        if results.pose_landmarks\
        else np.zeros(num_mp_lmks['pose']['total'])

face = np.array([[res.x, res.y, res.z]\
        for res in results.face_landmarks.landmark]).flatten()\
        if results.face_landmarks\
        else np.zeros(num_mp_lmks['face']['total'])

lhand = np.array([[res.x, res.y, res.z]\
        for res in results.pose_landmarks.landmark]).flatten()\
        if results.left_hand_landmarks\
        else np.zeros(num_mp_lmks['hand']['total'])

rhand = np.array([[res.x, res.y, res.z]\
        for res in results.pose_landmarks.landmark]).flatten()\
        if results.right_hand_landmarks\
        else np.zeros(num_mp_lmks['hand']['total'])


In [ ]:
len(pose)

In [ ]:
len(face)

In [ ]:
len(lhand)

In [ ]:
len(rhand)

In [171]:
# extract_keypoints(results)
# returns a single array with every single keypoint value in a frame
#results are the media pipe detection results

def extract_keypoints(results):
    if results.pose_landmarks:
        pose = np.array([[res.x, res.y, res.z, res.visibility]\
            for res in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(num_mp_lmks['pose']['total'])
    
    if results.face_landmarks:
        face = np.array([[res.x, res.y, res.z]\
            for res in results.face_landmarks.landmark]).flatten()
    else:
        face = np.zeros(num_mp_lmks['face']['total'])

    if results.left_hand_landmarks:
        lhand = np.array([[res.x, res.y, res.z]\
            for res in results.left_hand_landmarks.landmark]).flatten()
    else:
        lhand = np.zeros(num_mp_lmks['hand']['total'])

    if results.right_hand_landmarks:
        rhand = np.array([[res.x, res.y, res.z]\
            for res in results.right_hand_landmarks.landmark]).flatten()
    else:
        rhand = np.zeros(num_mp_lmks['hand']['total'])

    return np.concatenate([pose, face, lhand, rhand])

In [ ]:
extract_keypoints(results)[:10]

## 4. Set up folders for collection

In [172]:
# path for exported data, numpy arrays
DATA_PATH = os.path.join('ASL', 'MP_Data')

# actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou', 'boy', 'neutral', 'not_hotdog'])

# 25 videos worth of data per action
num_sequences = 25

#videos will be 30 frames in length
sequence_length = 30

In [116]:
# create folders
# 1 folder per action
# 1 folder per sequence
# 30 frames worth od data will be in each sequence folder

for action in actions:
    for sequence in range(num_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Collect keypoint values for training and testing

In [ ]:
# my test array
result_test = extract_keypoints(results)
result_test

In [ ]:
# save this test array to disk
np.save('test_results', result_test)

In [ ]:
# load the test array
np.load('test_results.npy')

In [119]:
cap = cv2.VideoCapture(2) # 2 is for logitech (near right USB-C port)
# set the mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    # loop through actions
    for action in actions:
        # loop through sequences (aka videos)
        for sequence in range(num_sequences):
            # loop through video length (aka sequence length)
            for frame_num in range(sequence_length):
                
                #read feed
                ret, frame = cap.read()
                
                #make detection
                image, results = mediapipe_detection(frame, holistic)
                
                #draw landmarks and connections
                draw_landmarks(image, results)
                
                #collection pauses and messaging
                if frame_num == 0:
                    cv2.putText(frame, ('STARTING {} IN 2 SECONDS!'.format(action)),
                                (120,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,150,0), 2, cv2.LINE_AA)
                    cv2.putText(frame, ('Collecting frames for {} Video Number {}'.format(action, sequence)),
                                (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,150), 1, cv2.LINE_AA)
                    # show to screen
                    cv2.imshow('OpenCV Feed', frame)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(frame, ('Collecting frames for {} Video Number {}'.format(action, sequence)),
                                (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,150), 1, cv2.LINE_AA)
                    # show to screen
                    cv2.imshow('OpenCV Feed', frame)
                
                #export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                #break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                # end video capture
    cap.release()
    cv2.waitKey(1)
    # close the video capture window(s)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [ ]:
cap.release()
cv2.waitKey(1)
# close the video capture window(s)
cv2.destroyAllWindows()
cv2.waitKey(1)

## 6. Pre-process data and create labels and features

__sequences__ (__videos__) are going to represent our __feature data__ (aka __x-data__)
__labels__ (is the video hello, thanks or iloveyou) represents our __y-data__

We are going to use our features (sequences or videos) to __train__ a model to represent the __relationship__ between the __features__ and the __labels__.

In [173]:
# import dependencies
from sklearn.model_selection import train_test_split
#helps us split up data for testing and training

from tensorflow.keras.utils import to_categorical
# to convert an np array of values to np array of 0s, 1s

In [ ]:
print(actions)

In [174]:
label_map = {label : num for num, label in enumerate(actions)}

In [175]:
label_map

{'hello': 0,
 'thanks': 1,
 'iloveyou': 2,
 'boy': 3,
 'neutral': 4,
 'not_hotdog': 5}

In [176]:
# bring in the saved data
sequences, labels = [],[] #empty arrays. Think of sequences as x-data, label as y-data
for action in actions:
    for seq in range(num_sequences):
        window = [] # all of the frames for this sequence
        for frame_num in range(sequence_length):
            result = np.load(os.path.join(DATA_PATH, action, str(seq), '{}.npy'.format(frame_num)))
            window.append(result)
        sequences.append(window)
        labels.append(label_map[action])

In [125]:
np.shape(sequences)

(150, 30, 1662)

In [126]:
np.shape(labels)

(150,)

In [177]:
x = np.array(sequences) # make a numpy array from sequences

In [178]:
sequences

[[array([ 0.44761017,  0.47765452, -0.66979498, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44744411,  0.4714874 , -0.75039613, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44662553,  0.46039873, -0.66388535, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44611204,  0.45804751, -0.78688538, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44559577,  0.45417997, -0.7885738 , ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44418621,  0.45084447, -0.5570969 , ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.4437657 ,  0.44827923, -0.52263421, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44377184,  0.44753912, -0.56917202, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44378859,  0.44800854, -0.67369437, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.44300613,  0.45088896, -0.76542908, ...,  0.        ,
       

In [179]:
x

array([[[ 4.47610170e-01,  4.77654517e-01, -6.69794977e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.47444111e-01,  4.71487403e-01, -7.50396132e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.46625531e-01,  4.60398734e-01, -6.63885355e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 4.36025918e-01,  4.35294658e-01, -7.31891334e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.37516421e-01,  4.35383976e-01, -6.59737229e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.39225465e-01,  4.35592711e-01, -6.70889974e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 4.40146863e-01,  4.35826242e-01, -7.23872900e-01, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 4.40512568e-01,  4.35447425e-01, -7.51652479e-01, ...,
          0.00000000e+00,  0.00000000e

In [130]:
np.shape(x)

(150, 30, 1662)

In [180]:
labels # it's a vector with values either 0, 1, or 2

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5]

In [181]:
np.shape(labels)

(150,)

In [182]:
# change (75,) label vector w values of 0,1,2 to (3,) category vector
# where 1 is either in cell[0], cell[1], or cell[2]
# basically a binary flag
# i.e. label value = 2 --> category value = [0, 0, 1]
y = to_categorical(labels).astype(int)

In [183]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
 

In [135]:
y[24]

array([1, 0, 0, 0, 0, 0])

In [136]:
y[49]

array([0, 1, 0, 0, 0, 0])

In [139]:
y[105]

array([0, 0, 0, 0, 1, 0])

In [140]:
np.shape(y)

(150, 6)

### split our data into training and testing
__train_test_split()__ will make a __random__ selection each time

In [184]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.06) #6% will be test data

In [185]:
y_test

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

In [145]:
len(x_test)

9

In [146]:
len(y_test)

9

## 7. Build and train LSTM neural network
__LSTM__: Long short-term memory

We're going to pass 30 frames with 1662 keyframes each.
We will then pre-process the result and extract the action.

In [186]:
# import required packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [187]:
# set up the log path to then view the log with TensorBoard
LOG_DIR = os.path.join('ASL', 'logs')
tb_callback = TensorBoard(log_dir = LOG_DIR)

<a id="model_rebuild"></a>
### Build or rebuild the model
This is the model's shape
When [loading a model](#load_model) from disk, make sure to:
1. run this model rebuild first
2. [compile](#compile_model) the model
3. then you can load [load the model](#load_model)

In [188]:
# instantiate the Sequential API model
model = Sequential()

# creating the LSTM layers
# 1st layer: 64 LSTM units, return sequence (True) so next layer can use it
# shape of each sequence (video) is 30 frames by 1662 keypoints (30,1662)
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))

# 2nd layer: 128 LSTM units
model.add(LSTM(128, return_sequences=True, activation='relu'))

#3rd layer: 64 LSTM units, won't be returning the sequence
model.add(LSTM(64, return_sequences=False, activation='relu'))

# creating the Dense layers
# 1st layer: 64 Densly connected Neural Network neurons
model.add(Dense(64, activation='relu'))

#2nd layer: 32 Dense NN neurons
model.add(Dense(32, activation='relu'))

# create the Actions layer
# 3 neural network units (actions.shape = (3,) and so actions.shape[0] = 3)
# choosing softmax because all three values in the model will add up to 1
model.add(Dense(actions.shape[0], activation='softmax'))

In [150]:
x.shape

(150, 30, 1662)

In [151]:
actions.shape

(6,)

In [152]:
actions.shape[0]  # 3 neural network units

6

In [153]:
#softmax result example
res = [.7, .2, .1]
#rounds to: [1, 0, 0]
#label inference result is "hello"
np.argmax(res) # "hello"

0

In [154]:
actions[np.argmax(res)]

'hello'

In [155]:
res2 = [.3, .6, .1]
np.argmax(res2) # "thanks"

1

In [156]:
actions[np.argmax(res2)]

'thanks'

<a id="compile_model"></a>
### compile the model
After you have defined the model's shape, you will need to compile the model.

When [loading a model](#load_model) from disk, make sure to:
1. run the [model rebuild](#model_rebuild) first
2. Run this model compiler
3. then you can load [load the model](#load_model)

In [189]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


### train the model

In [190]:
model.fit(x_train, y_train, epochs=400, callbacks=[tb_callback])

Epoch 1/400
5/5 [==============================] - 3s 73ms/step - loss: 3.5696 - categorical_accuracy: 0.1348
Epoch 2/400
5/5 [==============================] - 0s 67ms/step - loss: 2.0907 - categorical_accuracy: 0.1702
Epoch 3/400
5/5 [==============================] - 0s 65ms/step - loss: 1.7999 - categorical_accuracy: 0.1986
Epoch 4/400
5/5 [==============================] - 0s 67ms/step - loss: 1.7030 - categorical_accuracy: 0.3191
Epoch 5/400
5/5 [==============================] - 0s 69ms/step - loss: 1.7174 - categorical_accuracy: 0.3050
Epoch 6/400
5/5 [==============================] - 0s 65ms/step - loss: 1.6427 - categorical_accuracy: 0.3262
Epoch 7/400
5/5 [==============================] - 0s 66ms/step - loss: 1.5273 - categorical_accuracy: 0.4043
Epoch 8/400
5/5 [==============================] - 0s 65ms/step - loss: 1.5717 - categorical_accuracy: 0.3050
Epoch 9/400
5/5 [==============================] - 0s 65ms/step - loss: 1.4536 - categorical_accuracy: 0.3546
Epoch 10/4

5/5 [==============================] - 0s 73ms/step - loss: 0.0442 - categorical_accuracy: 0.9858
Epoch 148/400
5/5 [==============================] - 0s 70ms/step - loss: 0.0500 - categorical_accuracy: 0.9787
Epoch 149/400
5/5 [==============================] - 0s 86ms/step - loss: 0.0450 - categorical_accuracy: 0.9716
Epoch 150/400
5/5 [==============================] - 0s 84ms/step - loss: 0.0540 - categorical_accuracy: 0.9716
Epoch 151/400
5/5 [==============================] - 0s 77ms/step - loss: 0.0415 - categorical_accuracy: 0.9858
Epoch 152/400
5/5 [==============================] - 0s 83ms/step - loss: 0.0437 - categorical_accuracy: 0.9787
Epoch 153/400
5/5 [==============================] - 0s 70ms/step - loss: 0.0566 - categorical_accuracy: 0.9716
Epoch 154/400
5/5 [==============================] - 0s 68ms/step - loss: 0.0490 - categorical_accuracy: 0.9858
Epoch 155/400
5/5 [==============================] - 0s 70ms/step - loss: 0.0476 - categorical_accuracy: 0.9787
Epoch 

5/5 [==============================] - 0s 71ms/step - loss: 0.0258 - categorical_accuracy: 1.0000
Epoch 294/400
5/5 [==============================] - 0s 66ms/step - loss: 0.0207 - categorical_accuracy: 0.9929
Epoch 295/400
5/5 [==============================] - 0s 66ms/step - loss: 0.0199 - categorical_accuracy: 0.9929
Epoch 296/400
5/5 [==============================] - 0s 66ms/step - loss: 0.0205 - categorical_accuracy: 0.9929
Epoch 297/400
5/5 [==============================] - 0s 66ms/step - loss: 0.0103 - categorical_accuracy: 1.0000
Epoch 298/400
5/5 [==============================] - 0s 64ms/step - loss: 0.0129 - categorical_accuracy: 0.9929
Epoch 299/400
5/5 [==============================] - 0s 65ms/step - loss: 0.0143 - categorical_accuracy: 0.9929
Epoch 300/400
5/5 [==============================] - 0s 65ms/step - loss: 0.0260 - categorical_accuracy: 0.9929
Epoch 301/400
5/5 [==============================] - 0s 65ms/step - loss: 0.0465 - categorical_accuracy: 0.9858
Epoch 

In [191]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 6)                 198       
                                                                 
Total params: 596,774
Trainable params: 596,774
Non-tr

## 8. Make predictions
Make __predictions__ on __x_test__ and __compare__ them to the __correct answers__ in __y_test__

In [192]:
# this is our test data
x_test[:2]

array([[[ 0.45479107,  0.39945138, -0.48474866, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.44846323,  0.39991125, -0.6878612 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.44311053,  0.4001075 , -0.6850692 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.46685922,  0.40611938, -0.42988709, ...,  0.68579918,
          0.58842772,  0.02505593],
        [ 0.46726948,  0.40603712, -0.42447367, ...,  0.6828016 ,
          0.58604527,  0.03285921],
        [ 0.46750799,  0.40603599, -0.43220717, ...,  0.67939144,
          0.58335698,  0.03114768]],

       [[ 0.45634827,  0.396054  , -0.67959815, ...,  0.49308938,
          0.73083073, -0.00846376],
        [ 0.45382211,  0.39746407, -0.65490222, ...,  0.43348059,
          0.637245  , -0.01689984],
        [ 0.45020565,  0.39760143, -0.56066114, ...,  0.43590814,
          0.63397229, -0.02928158],
        ...,
        [ 0.45391923,  0.39587626, -0.54002208, ...,  

In [193]:
# use the model to predict what our test data is
res3 = model.predict(x_test)

1/1 [==============================] - 0s 343ms/step


In [194]:
# test results as category arrays
# each test unit result is an array of probabilities that add up to one
# the one with the highest probability is the prediction
res3

array([[1.1156309e-06, 1.7356961e-05, 6.4660009e-05, 2.0624504e-05,
        3.8862861e-05, 9.9985731e-01],
       [1.1504218e-12, 8.8443795e-09, 5.5867151e-09, 4.5846918e-10,
        8.4676790e-09, 1.0000000e+00],
       [2.9930452e-21, 2.1848464e-34, 4.9553935e-20, 2.9155636e-14,
        1.0000000e+00, 3.0481450e-21],
       [2.6271740e-01, 1.5788234e-04, 7.3681933e-01, 1.0928845e-05,
        2.6238911e-04, 3.2002899e-05],
       [1.3171765e-06, 1.0351552e-06, 2.7739222e-06, 1.8851244e-04,
        1.0604831e-05, 9.9979573e-01],
       [2.5457084e-07, 1.5123376e-06, 7.0357437e-06, 8.4433914e-06,
        6.9845869e-06, 9.9997568e-01],
       [7.0408154e-05, 1.4108931e-05, 6.1130524e-04, 4.1285362e-05,
        2.4959632e-05, 9.9923790e-01],
       [2.7063072e-14, 1.0000000e+00, 2.6511327e-15, 3.4348962e-21,
        1.1166566e-17, 8.4045133e-15],
       [1.8380261e-07, 6.7057073e-05, 9.9062270e-01, 4.8818816e-10,
        5.0353997e-06, 9.3049463e-03]], dtype=float32)

In [195]:
res3[0]

array([1.1156309e-06, 1.7356961e-05, 6.4660009e-05, 2.0624504e-05,
       3.8862861e-05, 9.9985731e-01], dtype=float32)

In [196]:
np.sum(res3[0])

0.99999994

In [197]:
actions

array(['hello', 'thanks', 'iloveyou', 'boy', 'neutral', 'not_hotdog'],
      dtype='<U10')

In [198]:
for i in range(len(res3)):
    print(actions[np.argmax(res3[i])])

not_hotdog
not_hotdog
neutral
iloveyou
not_hotdog
not_hotdog
not_hotdog
thanks
iloveyou


In [199]:
# match our predictions with the actual results:
#accurate!!
y_test

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

## 9. Save weights (aka the model)

In [200]:
model

In [215]:
# save the model I trained
model.save(os.path.join('ASL', 'model_2.h5'))

In [ ]:
# now I can delete the model
del model

<a id="load_model"></a>
### reload steps:
1. run the [model rebuild](#model_rebuild)
2. [compile](#compile) the data
3. reload model

In [32]:
model.load_weights(os.path.join('ASL', 'my_first_action_model.h5'))

In [202]:
model

## 10. Evaluation using confusion matrix and accuracy

In [ ]:
### TBD ###

## 11. Test in Real Time

In [203]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (10,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (10, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (50,100,100), 2, cv2.LINE_AA)
        
    return output_frame

In [208]:
# 1. Three new detection variables
sequence = [] #to collect our 30 frames. Once we get 30 frames we will start predicting
sentence = [] #concatenate our history of detections together
threshold = 0.8 #render results only if above a certain threshold

cap = cv2.VideoCapture(2) # 2 is for logitech (near right USB-C port)
# set the mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        
        #read feed
        ret, frame = cap.read()
        
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        
        #draw landmarks and connections
        draw_landmarks(image, results)
        
        # show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    # end the video capture setting
    cap.release()
    cv2.waitKey(1)
    # close the video capture window(s)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [205]:
np.expand_dims(x_test[0], axis=0)

array([[[ 0.45479107,  0.39945138, -0.48474866, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.44846323,  0.39991125, -0.6878612 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.44311053,  0.4001075 , -0.6850692 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.46685922,  0.40611938, -0.42988709, ...,  0.68579918,
          0.58842772,  0.02505593],
        [ 0.46726948,  0.40603712, -0.42447367, ...,  0.6828016 ,
          0.58604527,  0.03285921],
        [ 0.46750799,  0.40603599, -0.43220717, ...,  0.67939144,
          0.58335698,  0.03114768]]])

In [206]:
model.predict(np.expand_dims(x_test[0], axis=0))

1/1 [==============================] - 0s 21ms/step


array([[1.1156309e-06, 1.7357010e-05, 6.4659762e-05, 2.0624544e-05,
        3.8862934e-05, 9.9985731e-01]], dtype=float32)

In [214]:
# 1. Three new detection variables
sequence = [] #to collect our 30 frames. Once we get 30 frames we will start predicting
sentence = [] #concatenate our history of detections together
threshold = 0.5 #render results only if above a certain threshold

cap = cv2.VideoCapture(2) # 2 is for logitech (near right USB-C port)
# set the mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        
        #read feed
        ret, frame = cap.read()
        
        #make detection
        image, results = mediapipe_detection(frame, holistic)
        
        #draw landmarks and connections
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
        # 3. Vis logic
        if res[np.argmax(res)] > threshold: # checking if result is above our threshold
            if len(sentence) > 0: 
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

                    
        if len(sentence) > 5:
            sentence = sentence[-5:]
            
        cv2.rectangle(image, (0,0), (640,40), (245, 17, 16), -1)
        cv2.putText(image, ' {}'.format(sentence[-1]), (3,30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    # end the video capture setting
    cap.release()
    cv2.waitKey(1)
    # close the video capture window(s)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step


In [110]:
    # end the video capture setting
    cap.release()
    cv2.waitKey(1)
    # close the video capture window(s)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

-1